# <font color='#eb3483'> Evaluating Time Series </font>

Great so we have some time series models - how do we tell if they're any good? In this module we'll explore how to evaluate our forecasts.

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
sns.set(rc={'figure.figsize':(16,5)})

import warnings
warnings.simplefilter("ignore")
%matplotlib inline

## <font color='#eb3483'> Building oour forecast </font>


Lets create the same SARIMAX model to predict airline passengers as we did before, however, instead of following the sklearn-like model syntax, we will use statsmodels api which is a way to use multiple kinds of statistical models with a similar syntax.

In [ ]:
import statsmodels.api as sm

import utils 

In [ ]:
airlines = utils.load_airline_data()

In [ ]:
airlines = airlines[:'1957']

In [ ]:
model = sm.tsa.statespace.SARIMAX(airlines,          
                          order=(0, 1, 1),           
                          seasonal_order=(1, 1, 1, 12)
                                 ) 

In [ ]:
results = model.fit()

In [ ]:
# We want to make 30 steps out of time 
train_up_to_step = len(airlines) - 30

In [ ]:
pred = results.get_prediction(start=train_up_to_step, 
                              dynamic=False)

pred

## <font color='#eb3483'> Time Series Evaluation Metrics </font> 

There are many metrics we can use to evaluate time series.

A common metric for evaluating time series is the RMSE, and r-squared, $R^{2}$, which we have seen already.

R2 is generally used only to validate the test set results. 

_Optional bit: The demonstration of why $R^{2}$ is beyond the scope here, but intuitive enough: a really complex model can overfit and get a high $R^{2}$, so if we use it as a metric to optimize when choosing the model we're incentivizing it to overfit._

Let's evaluate some test set results with R2: 

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
y_pred = pred.predicted_mean
y_true = airlines.iloc[train_up_to_step::]

In [ ]:
r2_score(y_pred, y_true)

### <font color='#eb3483'> AIC  </font>

As we mentioned, $R^{2}$ is limited when applied to the training set. This is where AIC is a better choice. 

AIC (Akaike information criterion) is a metric that will simutaneously measure how well the model fits the data, but will control for how complex the model is. If the model is very complex, the expectation oh how well it must fit the data will also go up. It is therefore useful for comparing models.  The lower the AIC, the better performance.

If you (for some weird reason) feel compelled to calculate it by hand, [this post](https://stats.stackexchange.com/questions/87345/calculating-aic-by-hand-in-r?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa) explains how to do so. Then again, it's sunny and beautiful outside, and Statsmodel has got your back. 


It's so useful (and hard to calculate), that statsmodels calculate it out of the box

In [ ]:
results.aic

### <font color='#eb3483'> MAPE </font>

Mape stands for [Mean Absolute Percentage Error](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error), which is calculated similarly as the Mean Absolute Error, but using the percentage difference between prediction and true target values.

It is usually presented as a percentage. For example, if the MAPE is 5, on average, the forecast is off by 5%. 

Its equation is:
$$
MAPE = \frac{100\%}{n}\sum_{i=1}^{n}\left |\frac{\hat{y}_i -y_i}{y_i}\right|
$$

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_pred- y_true) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(y_pred, y_true)

## <font color='#eb3483'> Hyper parameter Optimization </font>

Now, we have been using some very specific parameters: 

> p = 0  
> d = 1  
> q = 1  
> P = 1  
> D = 1  
> Q = 1  
> S = 12   

I discovered these parameters by doing one of the following: 
> A) _~~developing a strong intuition about how statsmodels work and about the trends in the airline industry~~_   
> B) throwing a hyper parameter optimizer at the problem and making myself a nice cup of tea while it ran. 

Since the SARIMAX parameter's space is fairly limited, we can do a very simple hyperparameter optimization

In [ ]:
p = d = q = P = D = Q = range(0, 2)   
S = [7, 12]

In [ ]:
import itertools

params_combinations = list(itertools.product(p, d, q, P, D, Q, S))

In [ ]:
params_combinations

In [ ]:
def get_aic(series_, params):
    # extract the params 
    p = params[0] 
    d = params[1] 
    q = params[2] 
    P = params[3]
    D = params[4] 
    Q = params[5]
    S = params[6]
    
    # fit a model with those params 
    model = sm.tsa.statespace.SARIMAX(series_,
                                      order=(p, d, q),
                                      seasonal_order=(P, D, Q, S),
                                      enforce_stationarity=False,
                                      enforce_invertibility=False)
    
    # fit the model
    results = model.fit()
    
    # return the aic 
    return results.aic

In [ ]:
%%time 

aic_scores = {}
params_index = {}

for i, param_set in enumerate(params_combinations):
    aic = get_aic(airlines, param_set) 
    aic_scores[i] = aic
    params_index[i] = param_set

In [ ]:
temp = pd.DataFrame(params_index).T
temp.columns = ['p', 'd', 'q', 'P', 'D', 'Q', 'S']
temp['aic'] = pd.Series(aic_scores)
temp.sort_values('aic').head()

<hr>

## <font color='#eb3483'> Using season and exogenous variables </font>

So far, we have been using only the endogenous variable to create predictions.

Also, the time series we used are ... kind of easy. Highly seasonal and periodic, eventhough the variance might increase over time. But they are easy. 

So, what about making things a "little bit" harder? Like, for example, predicting the US GDP Growth. If you want additional insights on each of the time series of this dataset, check this page at [thebalance.com](https://www.thebalance.com/components-of-gdp-explanation-formula-and-chart-3306015).

In [ ]:
data = pd.read_csv('data/US_Production_Q_Data_Growth_Rates.csv')

data.Year = pd.to_datetime(data.Year)

data = data.set_index('Year')

data.head(10)

In [ ]:
data['GDP Growth'].plot();

Can you see any seasonality? Me neither. But you might notice that, from time to time, there is a big drop in GDP Growth (to negative values). That is related to economical cycles. Hmmm...cycles...maybe a cyclical component is present?

**ALSO**, Remember the 2007 crisis? Well, it is no surprise that we had the biggest recession near ~2009. 

Let's plot all time series together to see if there is a pattern

In [ ]:
data.plot();

Well...maybe it wasn't a good idea. Let's instead make several plots, one for each pair (GDP Growth, OTHER TIME SERIES)

In [ ]:
for gdp, other in itertools.product(['GDP Growth'], data.drop('GDP Growth', axis=1).columns): 
    data[[gdp, other]].plot()
    sns.mpl.pyplot.show()

By visual inspection, we that Consumption Growth and Labor Growth follow GDP Growth very clearly (which makes sense, since consumption is one of the measures that compute GDP). So, one reasonable hypothesis is: those two time series are highly predictive for GDP Growth. 

Instead of trying to find the best model using grid search, we will explore the effect of each exogenous variable and model parameter in the forecast. 

First, let's prepare the train test split and, also, add some cyclical features for month and decade

In [ ]:
X = data.drop('GDP Growth', axis=1)
X['month (cosine)'] = np.cos(2 * np.pi * X.index.month/ 12)
X['month (sin)'] = np.sin(2 * np.pi * X.index.month/ 12)
X['decade (cosine)'] = np.cos(2 * np.pi * (X.index.year % 10) / 10)
X['decade (sine)'] = np.sin(2 * np.pi * (X.index.year % 10) / 10)

y = data['GDP Growth']

train_percentage = 0.5
train_size = int(X.shape[0] * train_percentage)

X_train, y_train = X.iloc[:train_size], y.iloc[:train_size]
X_test, y_test = X.iloc[train_size:], y.iloc[train_size:]

Now we can actually create our own sklearn compatible sarima model for performing a grid search

In [ ]:
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SARIMAXWrapper(BaseEstimator, RegressorMixin):
    def __init__(self,  
                 p, d, q, P, D, Q, S, 
                 trend, 
                 exog,
                 forecast_steps=None, 
                 data_train=None,
                 data_test=None):

        self.p = p
        self.d = d
        self.q = q
        self.trend = trend
        self.P = P
        self.D = D
        self.Q = Q
        self.S = S
        self.exog = exog
        self.data_train = data_train
        self.data_test = data_test
        self.forecast_steps = forecast_steps
        
    def fit(self, X, y):
        try:
            if self.exog in self.data_train.columns:
                exog_train = self.data_train[self.exog]  
            else:
                exog_train = None
            self.model_ = sm.tsa.SARIMAX(y, 
                                 exog=exog_train, # the exogeneous variable
                                 order=(self.p, self.d, self.q), # original arima p,d,q
                                 seasonal_order=(self.P, self.D, self.Q, self.S) # Seasonal p,d,q
                            )
            self.results_ = self.model_.fit(maxiter=10, 
                                  trend=self.trend)
                            
        except:
            self.results_ = (sm.tsa.SARIMAX(y, 
                                 order=(0,0,0), 
                                 seasonal_order=(0,0,0, 0))
                             .fit(maxiter=1, 
                                  trend=self.trend)
                            )
    def predict(self, *args, **kwargs):
        if self.exog in data.columns: 
            exog_test = (self
                     .data_test[self.exog]
                     .iloc[:self.forecast_steps]
                     .values
                     .reshape(self.forecast_steps, 1)
                    )
        else:
            exog_test = None
        return self.results_.get_forecast(
            steps=self.forecast_steps, 
            exog=exog_test
        ).predicted_mean
    
    def score(self, *args, **kwargs):
        return -1. * self.results_.aic

In [ ]:
m = SARIMAXWrapper(
              data_train=X_train,
              data_test=X_test,
              exog="Investment Growth", 
              p=1, d=1, q=1,
              trend="nc", 
              P=1, D=1, Q=1, S=1,
            forecast_steps=10
)

In [ ]:
m.fit(None, y_train)

In [ ]:
m.predict()

In [ ]:
m.score()

In [ ]:
m.get_params()

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import random

search_dist = {
    "p": randint(0,6),
    "d": randint(0,6),
    "q": randint(0,6),
    "P": randint(0,6),
    "D": randint(0,6),
    "Q": randint(0,6),
    "S": randint(0,13),
    "exog": list(data.drop('GDP Growth', axis=1).columns) + [None]
}

In [ ]:
r = RandomizedSearchCV(m, search_dist, n_iter=100, n_jobs=-1)

In [ ]:
r.fit(X_train, y_train)

We can get the results of the search and find the optimal hyperparameters with exogeneus variable included:

In [ ]:
(pd.DataFrame(r.cv_results_)
 .sort_values(by="mean_test_score", ascending=False)
 .head(10)
 [[
     "param_p",
     "param_d",
     "param_q",
     "param_P",
     "param_D",
     "param_Q",
     "param_S",
     "param_exog",
     "mean_test_score"
  ]]
)

In [ ]:
r.best_params_

In [ ]:
best_model = sm.tsa.SARIMAX(y, 
                            exog=None, # the exogeneous variable
                            order=(1, 1, 1), # original arima p,d,q
                            seasonal_order=(3, 0, 0, 4) # Seasonal p,d,q
)
results = best_model.fit()
results.aic

In [ ]:
def plot_predictions(series_, pred_):
    
    """ 
    Utility function to plot time series predictions with Confidence intervals
    """
    mean_predictions_ = pred_.predicted_mean
    pred_ci_ = pred_.conf_int()  
    series_.plot(label='observed')
    mean_predictions_.plot(label='predicted', 
                           alpha=.7)

    sns.mpl.pyplot.fill_between(pred_ci_.index,
                     pred_ci_['lower GDP Growth'],
                     pred_ci_['upper GDP Growth'], 
                     color='k', 
                     alpha=.2)

    sns.mpl.pyplot.ylim([-5, 15])
    sns.mpl.pyplot.legend()
    sns.mpl.pyplot.show()

In [ ]:
# We want to make 30 steps out of time 
train_up_to_step = len(y) - 10

# remember the dynamic argument? Well, we'll use the first 30 steps to train
pred = results.get_prediction(start=y.index.min(),  
                              dynamic=train_up_to_step)                     
    
plot_predictions(series_=y, pred_=pred)